In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")
import regex
import openpyxl
from openpyxl import load_workbook, formatting, styles
import os
import pycountry
pd.set_option('display.max_columns', None)
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl import workbook
from openpyxl.utils import get_column_letter
import customtkinter 
import re

In [2]:

# Function to choose internal template
def choose_internal_template():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=(("Excel files", ["*.xlsx","*.xlsm"]), ("All files", "*.*")))

# Function to choose customer template
def choose_customer_template():
    global c_file_path
    c_file_path = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))

def choose_gateway_file():
    global g_file_path
    g_file_path = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*"))) 
    
def choose_gcr_template():
    global gcr_file_path
    gcr_file_path = filedialog.askopenfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
        

def process_customer_template(customer_data, num_rows):
    new_columns = []
    flattened_columns = []

    for i in range(num_rows):
        new_columns.append(customer_data.iloc[i, :].fillna('').astype(str))

    customer_data.columns = pd.MultiIndex.from_frame(pd.concat(new_columns, axis=1))

    main_heading = ""
    temp = {}
    for column in customer_data.columns:
        main_column = column[0]
        sub_columns = []

        if main_column != "":
            main_heading = main_column
            temp = {}

        for j in range(1, num_rows):
            if column[j] != "":
                temp[j] = column[j]
            elif j in temp:
                column = list(column)
                column[j] = temp[j]
                column = tuple(column)
            sub_columns.append(column[j])

        if main_heading == "":
            joined_column = "_".join(sub_columns)
        else:
            joined_column = f'{main_heading}_{("_".join(sub_columns))}'
        flattened_columns.append(joined_column)

    customer_data.columns = flattened_columns
    
    customer_data = customer_data.iloc[num_rows:]
    
    customer_data.reset_index(drop=True, inplace=True)
    return customer_data



def column_treatment(columns):
    
    last_non_unnamed = '' if 'unnamed' in columns[0].lower() else columns[0]  # Initializing last named column
    new_columns = []  # Initializing new list of modified column names
    new_column = ''
    for column in columns:
        if 'unnamed' in column.lower():
            new_column = last_non_unnamed  # Modifying column name to match last named column
        else:
            new_column = column
            last_non_unnamed = new_column  # Updating last named column
        new_columns.append(new_column)

    return new_columns

def template_cleaning(df):
    df.columns = column_treatment(df.columns)
    df.iloc[0] = df.iloc[0].astype(str)
    df.columns = ['_'.join(i) for i in zip(df.columns.get_level_values(0).tolist(), df.iloc[0, :].replace(np.nan, '').tolist())]
    df = df.drop(index=0)
    df.reset_index(drop=True, inplace=True)

    return df


def transfer_data():
    if num_rows_entry.get().strip() == '':
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return
    
    try:
        num_rows = int(num_rows_entry.get())
    except ValueError:
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return
    #internal_column = internal_column_dropdown.get()
    #customer_column = customer_column_dropdown.get()
    global internal_column_dropdowns
    global processed_data
    global internal_template
    global gcr_og_data
    global gcr_dest_data
    
   
    customer_file_path = c_file_path
    num_rows = int(num_rows_entry.get())
    skip_rows = int(skip_rows_entry.get())
    sheet_name = sheet_entry.get()
    
    gcr_og_data = pd.read_excel(gcr_file_path, skiprows=0, sheet_name='Origin')
    gcr_dest_data=pd.read_excel(gcr_file_path, skiprows=0, sheet_name='Destination')

    if skip_rows > 0:
        skip_rows = skip_rows - 1
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows, sheet_name=sheet_name)
        else:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows)
    else:
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, sheet_name=sheet_name, header=None)
        else:
            customer_data = pd.read_excel(customer_file_path, header=None)

    processed_data = process_customer_template(customer_data, num_rows)
    #customer_column_dropdown['values'] = list(processed_data.columns)

    if file_path:
        internal_template = pd.read_excel(file_path, skiprows=2)
        internal_template = template_cleaning(internal_template)
        print("Internal template loaded successfully!")
        #internal_column_dropdown['values'] = list(internal_template.columns)

        # Additional functionality - List internal template column names and add dropdowns
       
        internal_columns = list(internal_template.columns)
        
#         frame_1 = customtkinter.CTkFrame(master=window)
#         frame_1.grid(row=6, column=0, padx=40, pady=20)

#         frame_2 = customtkinter.CTkScrollableFrame(master=frame_1, orientation="vertical", label_text="CTkScrollableFrame",width=580, height=250)
#         frame_2.grid(row=7, column=0, padx=40, pady=20)

        internal_column_dropdowns = []
        
        def clear_option_menu(menu_var, default_value):
            menu_var.set(default_value)
            
        remove_b=[]
        o_vars = []
        #options = customtkinter.StringVar(frame_2)
        for i, col in enumerate(internal_columns):
            options = customtkinter.StringVar(frame_2)
            options.set("")
            label = customtkinter.CTkLabel(frame_2, text=col)
            label.grid(row=i, column=0, sticky="w",padx=8, pady=10)          
            dropdown = customtkinter.CTkOptionMenu(frame_2, width=50,variable=options,values=list(processed_data.columns))           
            #dropdown.set("")
            #dropdown.bind('<<ComboboxSelected>>')
            dropdown.grid(row=i, column=2, sticky="w", padx=0, pady=10) 
            o_vars.append(options)
            
            internal_column_dropdowns.append(dropdown)
            
        for i, col in enumerate(internal_columns):
            b3 = customtkinter.CTkButton(frame_2, text='Remove' , command=lambda i=i: clear_option_menu(o_vars[i], ""))  
            b3.grid(row=i,column=1)                
            


       
    else:
        print("Please select an internal template.")
    pass

         

In [3]:
def transfer_data_2():
    if num_rows_entry_.get().strip() == '':
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return
    
    try:
        num_rows = int(num_rows_entry_.get())
    except ValueError:
        tk.messagebox.showerror("Error", "Please enter a valid number of rows.")
        return
    #internal_column = internal_column_dropdown.get()
    #customer_column = customer_column_dropdown.get()
    global internal_column_dropdowns
    global processed_data
    global internal_template
   
    
   
    customer_file_path = c_file_path
    num_rows = int(num_rows_entry_.get())
    skip_rows = int(skip_rows_entry_.get())
    sheet_name = sheet_entry_.get()
    
    
    if skip_rows > 0:
        skip_rows = skip_rows - 1
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows, sheet_name=sheet_name)
        else:
            customer_data = pd.read_excel(customer_file_path, skiprows=skip_rows)
    else:
        if sheet_name:
            customer_data = pd.read_excel(customer_file_path, sheet_name=sheet_name, header=None)
        else:
            customer_data = pd.read_excel(customer_file_path, header=None)
         
        

    processed_data = process_customer_template(customer_data, num_rows)
    processed_data.iloc[:,:]=''
    #customer_column_dropdown['values'] = list(processed_data.columns)

    if file_path:
        internal_template = pd.read_excel(file_path, skiprows=2)
        internal_template = template_cleaning(internal_template)
        print("Internal template loaded successfully!")
        #internal_column_dropdown['values'] = list(internal_template.columns)

        # Additional functionality - List internal template column names and add dropdowns
       
        internal_columns = list(internal_template.columns)
        
#         frame_1 = customtkinter.CTkFrame(master=window)
#         frame_1.grid(row=6, column=0, padx=40, pady=20)

#         frame_2 = customtkinter.CTkScrollableFrame(master=frame_1, orientation="vertical", label_text="CTkScrollableFrame",width=580, height=250)
#         frame_2.grid(row=7, column=0, padx=40, pady=20)

        internal_column_dropdowns = []
        
        def clear_option_menu(menu_var, default_value):
            menu_var.set(default_value)
            
        remove_b=[]
        o_vars = []
        #options = customtkinter.StringVar(frame_2)
        for i, col in enumerate(processed_data.columns):
            options = customtkinter.StringVar(frame_2_)
            options.set("")
            label = customtkinter.CTkLabel(frame_2_, text=col)
            label.grid(row=i, column=0, sticky="w",padx=8, pady=10)          
            dropdown = customtkinter.CTkOptionMenu(frame_2_, width=50,variable=options,values=list(internal_columns))           
            #dropdown.set("")
            #dropdown.bind('<<ComboboxSelected>>')
            dropdown.grid(row=i, column=2, sticky="w", padx=0, pady=10) 
            o_vars.append(options)
            
            internal_column_dropdowns.append(dropdown)
            
        for i, col in enumerate(processed_data.columns):
            b3 = customtkinter.CTkButton(frame_2_, text='Remove' , command=lambda i=i: clear_option_menu(o_vars[i], ""))  
            b3.grid(row=i,column=1)                
            


       
    else:
        print("Please select an internal template.")
    pass


In [4]:
def country_house_Column(x):
    """This function takes a pandas DataFrame as input and returns a concatenated string of origin port and 
    destination port if both of them are not null, otherwise returns NaN.
    
    Args:
    x (DataFrame): A pandas DataFrame which should have columns named 'origin port' and 'destination port'
    
    Returns:
    str or NaN: A concatenated string of origin port and destination port or NaN
    """
    
    if ((x['origin port'] != 'nan') & (x['destination port'] != 'nan')   ):
        return str(x['origin port']) + " " + str(x['destination port'])
    
    else:
        return np.nan
    pass

In [5]:
def gcr(df,df1,df2):
#     df.columns = column_treatment(df.columns)  
#     df.iloc[0] = df.iloc[0].astype(str)
#     df.columns = ['_'.join(i) for i in zip(df.columns.get_level_values(0).tolist(), df.iloc[0,:].replace(np.nan,'').tolist())]
    #df = df.drop(index = 0)
    #df = df.drop(index = 1)
    df['Origin  _Origin Gateway'] = df['Origin  _Origin Gateway'].astype(str)
    df1.columns = column_treatment(df1.columns)  
    df1.iloc[0] = df1.iloc[0].astype(str)
    df1.columns = ['_'.join(i) for i in zip(df1.columns.get_level_values(0).tolist(), df1.iloc[0,:].replace(np.nan,'').tolist())]
    df1 = df1.drop(index = 0)
    df1.drop_duplicates(subset=['Information_Origin Gateway Code'],keep='first',inplace=True)
    df1['Information_Origin Gateway Code'] = df1['Information_Origin Gateway Code'].astype(str)
    
    #Destination 
    df2.columns = column_treatment(df2.columns)  
    df2.iloc[0] = df2.iloc[0].astype(str)
    df2.columns = ['_'.join(i) for i in zip(df2.columns.get_level_values(0).tolist(), df2.iloc[0,:].replace(np.nan,'').tolist())]
    df2 = df2.drop(index = 0)
    df2.drop_duplicates(subset=['Information_Destination Gateway Code'],keep='first',inplace=True)
    df2['Information_Destination Gateway Code'] = df2['Information_Destination Gateway Code'].astype(str)
 
    df3 = pd.merge(df, df1, left_on = 'Origin  _Origin Gateway', right_on = 'Information_Origin Gateway Code', how = 'left')
    df3 = pd.merge(df3, df2, left_on = 'Destination  _Destination Gateway', right_on = 'Information_Destination Gateway Code', how = 'left')
    df3.to_clipboard()
    df3[' OTHER MANDATORY ORIGIN CHARGES_MIN']=df3[' OTHER MANDATORY ORIGIN CHARGES_MIN'].fillna(df3.pop('Terminal Handling_Min _x'))
    df3[' OTHER MANDATORY ORIGIN CHARGES_PER KG']=df3[' OTHER MANDATORY ORIGIN CHARGES_PER KG'].fillna(df3.pop('Terminal Handling_Per KG_x'))
    df3['ORIGIN AIRLINE HANDLING/SCREENING_MIN'] = df3['ORIGIN AIRLINE HANDLING/SCREENING_MIN'].fillna(df3.pop('Airline Handling/ Carryover Transfer_Min_x'))
    df3['ORIGIN AIRLINE HANDLING/SCREENING_PER KG'] = df3['ORIGIN AIRLINE HANDLING/SCREENING_PER KG'].fillna(df3.pop('Airline Handling/ Carryover Transfer_Per KG_x'))
    df3['AIRPORT TRANSFER FEE_MIN'] = df3['AIRPORT TRANSFER FEE_MIN'].fillna(df3.pop('Airport Transfer Fee_Min_x'))
    df3['AIRPORT TRANSFER FEE_PER KG'] = df3['AIRPORT TRANSFER FEE_PER KG'].fillna(df3.pop('Airport Transfer Fee_Per KG_x'))
    df3['ORIGIN DOCUMENTATION FEE_PER SHPT'] = df3['ORIGIN DOCUMENTATION FEE_PER SHPT'].fillna(df3.pop('Documentation Fee_Per SHPT_x'))
    df3[' EXPORT CUSTOMS CLEARANCE_CHARGE PER SHPT'] = df3[' EXPORT CUSTOMS CLEARANCE_CHARGE PER SHPT'].fillna(df3.pop('Export Customs Clearance_Per SHPT'))
    df3[' OTHER MANDATORY ORIGIN CHARGES.1_PER SHPT'] = df3[' OTHER MANDATORY ORIGIN CHARGES.1_PER SHPT'].fillna(df3.pop('AMS/ENS/MANIFEST_Per SHPT_x'))

    #DESTN
    df3['AIRPORT TRANSFER FEE.1_MIN'] = df3['AIRPORT TRANSFER FEE.1_MIN'].fillna(df3.pop('Airport Transfer Fee_Min_y'))
    df3['AIRPORT TRANSFER FEE.1_PER KG'] = df3['AIRPORT TRANSFER FEE.1_PER KG'].fillna(df3.pop('Airport Transfer Fee_Per KG_y'))
    df3['DEST. DOCUMENTATION FEE         _PER SHPT'] = df3['DEST. DOCUMENTATION FEE         _PER SHPT'].fillna(df3.pop('Documentation Fee_Per SHPT_y'))
    df3['IMPORT CUSTOMS CLEARANCE _PER SHPT'] = df3['IMPORT CUSTOMS CLEARANCE _PER SHPT'].fillna(df3.pop('Import Customs Clearance_per SHPT'))
    df3[' OTHER MANDATORY DESTINATION CHARGES_MIN'] = df3[' OTHER MANDATORY DESTINATION CHARGES_MIN'].fillna(df3.pop('Terminal Handling_Min _y'))
    df3[' OTHER MANDATORY DESTINATION CHARGES_PER KG'] = df3[' OTHER MANDATORY DESTINATION CHARGES_PER KG'].fillna(df3.pop('Terminal Handling_Per KG_y'))
    df3 = df3.loc[:, :'Dest Remarks_nan']
    
    
    return df3
    



In [6]:
def transfer_all_data_2():
    for i, col_dropdown in enumerate(internal_column_dropdowns):
        customer_column = processed_data.columns[i]
        internal_column = col_dropdown.get()
        if internal_column and customer_column:
             processed_data[customer_column]=internal_template[internal_column]
    print("Transfer successful")
    pass

In [7]:
def download_excel_2():
    file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    new_columns = [tuple(col.split('_')) for col in  processed_data.columns]
    processed_data.columns = pd.MultiIndex.from_tuples(new_columns)
    processed_data.dropna(how='all',inplace=True)   
    processed_data.reset_index(drop=True, inplace=True)
    l=len(processed_data)
    index=pd.Index(range(l),name='index')
    processed_data.index=index
    processed_data.to_excel(file_path)
    print("Excel file saved successfully!")
    pass

In [8]:
def transfer_all_data():
    for i, col_dropdown in enumerate(internal_column_dropdowns):
        internal_column = internal_template.columns[i]
        customer_column = col_dropdown.get()
        if internal_column and customer_column:
            internal_template[internal_column] = processed_data[customer_column]
    print("Transfer successful")
    pass

def truck_df_treatment():
    states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
    global gcr_data
    global internal_template
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None) 
    df = internal_template
    gateway = pd.read_excel(g_file_path)
    df1 = gateway
    #print(df)
    #df.columns = column_treatment(df.columns)
    #df.reset_index(inplace = True)
    #df.iloc[0] = df.iloc[0].astype(str)

    #df.columns = ['_'.join(i) for i in zip(df.columns.get_level_values(0).tolist(), df.iloc[0,:].replace(np.nan,'').tolist())]
    #df.rename( columns = {"Lane Information_Lane ID" : 'Lane'}, inplace = True)
    df['Origin  _Origin City'] = df['Origin  _Origin City'].str.upper()
    df['Origin  _Origin State'] = df['Origin  _Origin State'].str.upper()
    df['Destination  _Destination City'] = df['Destination  _Destination City'].str.upper()
    df['Destination  _Destination State'] = df['Destination  _Destination State'].str.upper()
    df['Origin  _Origin State'] = df['Origin  _Origin State'].map(states)
    df['Destination  _Destination State'] = df['Destination  _Destination State'].map(states)
    #df['Origin  _Origin Gateway'] = df['Origin  _Origin Gateway'].str.replace("Origin Gateway -", "")
    #df['Destination  _Destination Gateway'] = df['Destination  _Destination Gateway'].str.replace("Destination Gateway -", "")
    #df.drop_duplicates('Lane', keep='first', inplace=True)
    #df = df.drop(df.index[-1])
    #df = df.drop(df.index[-1])
    #df = df.drop(df.index[0])
    df['Destination  _Destination Country'].replace('USA', 'US', inplace=True)
    df['Origin  _Origin Country'].replace('USA', 'US', inplace=True)
    pattern=r'([A-Z]){3}'
    #origin gateway correction(removing keyword gateway)       
    df['Origin  _Origin Gateway']=df['Origin  _Origin Gateway'].astype(str).apply(lambda x: re.search(pattern,x)[0] if re.search(pattern,x) else x)
    
        
    #destination gateway keyword removing
    df['Destination  _Destination Gateway']=df['Destination  _Destination Gateway'].astype(str).apply(lambda x: re.search(pattern,x)[0] if re.search(pattern,x) else x)
   
    

   

    # Origin gateway with state
    df1_states = df1[['State', 'Gateway']]
    df1_states['State'] = df1_states['State'].str.upper()
    df1_states.drop_duplicates('State', keep='last', inplace=True)
    df = pd.merge(df, df1_states, left_on='Origin  _Origin State', right_on='State', how='left')
    df['Gateway'] = df['Gateway'].replace('SUB', np.nan)
    df['Origin  _Origin Gateway'].fillna(df['Gateway'], inplace=True)
    #df['Origin  _Origin Gateway'].fillna('', inplace=True)
    del df['State']
    del df['Gateway']

    # Origin gateway with city
    df1_city = df1[[' City/Airport ', 'Gateway']]
    df1_city[' City/Airport '] = df1_city[' City/Airport '].str.upper()
    df1_city.drop_duplicates(' City/Airport ', keep='last', inplace=True)
    df = pd.merge(df, df1_city, left_on='Origin  _Origin City', right_on=' City/Airport ', how='left')
    df['Origin  _Origin Gateway'].fillna(df['Gateway'], inplace=True)
    df['Origin  _Origin Gateway'].fillna('', inplace=True)
    #print(df)
    del df[' City/Airport ']
    del df['Gateway']

    # Destination gateway using states
    df1_states = df1[['State', 'Gateway']]
    df1_states['State'] = df1_states['State'].str.upper()
    df1_states.drop_duplicates('State', keep='last', inplace=True)
    df = pd.merge(df, df1_states, left_on='Destination  _Destination State', right_on='State', how='left')
    df['Gateway'] = df['Gateway'].replace('SUB', np.nan)
    df['Destination  _Destination Gateway'].fillna(df['Gateway'], inplace=True)
    del df['State']
    del df['Gateway']

    # Destination Gateway using city
    df1_city = df1[[' City/Airport ', 'Gateway']]
    df1_city[' City/Airport '] = df1_city[' City/Airport '].str.upper()
    df1_city.drop_duplicates(' City/Airport ', keep='last', inplace=True)
    df = pd.merge(df, df1_city, left_on='Destination  _Destination City', right_on=' City/Airport ', how='left')
    df['Destination  _Destination Gateway'].fillna(df['Gateway'], inplace=True)
    df['Destination  _Destination Gateway'].fillna('', inplace=True)
    del df[' City/Airport ']
    del df['Gateway']

    # Region
    #del df['Origin  _Origin Region']
    #del df['Destination  _Destination Region']

    df1_region = df1[['Region', 'Country']]
    df1_region.drop_duplicates('Country', keep='last', inplace=True)

    # Origin Region
    df = pd.merge(df, df1_region, left_on='Origin  _Origin Country', right_on='Country', how='left')
    df.rename(columns={'Region': 'Origin Region'}, inplace=True)
    del df['Country']

    df = pd.merge(df, df1_region, left_on='Destination  _Destination Country', right_on='Country', how='left')
    df.rename(columns={'Region': 'Destination Region'}, inplace=True)
    del df['Country']

    # Country House column
    df['Origin  _Origin Gateway'] = df['Origin  _Origin Gateway'].astype(str)
    df['Destination  _Destination Gateway'] = df['Destination  _Destination Gateway'].astype(str)
    df['Origin  _Origin Gateway'].replace('no service', 'nan', inplace=True)
    df['Destination  _Destination Gateway'].replace('no service', 'nan', inplace=True)
    df['origin port'] = df['Origin  _Origin Gateway'].copy()
    df['destination port'] = df['Destination  _Destination Gateway'].copy()
    df['Lane Information_Country House'] = df.apply(lambda x: country_house_Column(x), axis=1)
    del df['origin port']
    del df['destination port']
    #df.loc[df["Lane Information_Country House"].isnull(), 'Country House?'] = 'No'
    #df['Country House?'].fillna('Yes', inplace=True)
    #df.loc[df['Lane Information_Country House'].notnull(), 'Country House?'] = 'Yes'
    #df.loc[df['Lane Information_Country House'].isnull(), 'Country House?'] = 'No'
    #if df['Lane Information_Country House'] = " " df['Country House?'] = 'No' else df['Country House?'] = 'Yes'
    column_length = df['Origin  _Origin Country'].count()
    df = df.iloc[:column_length, :]
    for index in range(df.shape[0]):
        value = df.loc[index, 'Lane Information_Country House']
        if value.strip() == '':
            df.at[index, 'Country House?'] = 'No'
        else:
            df.at[index, 'Country House?'] = 'Yes'
    print("Gateway Data Transfer Successful!")
    internal_template = df
    gcr_data=gcr(df,gcr_og_data,gcr_dest_data)
    gcr_data=gcr_data.round(2)




In [9]:
def summary_tables():
    # Prompt user for the directory to save the Excel file
    save_directory = input("Please enter the directory to save the Excel file: ")

    # Generate the file name based on the function name
    file_name = "summary_tables.xlsx"

    # Concatenate the directory and file name to create the save path
    save_path = save_directory + "/" + file_name
    
    df = internal_template
    df1 = df.copy()
    df1=df1.rename(columns={'Lane Information_Country House':'Country House','Lane Information_Lane ID':'No of lanes','Shipment Information_Chargeable KGS':'Sum of Chargeable_Weight Kgs','Shipment Information_# of Shipments':'No of shipments','Origin  _Origin Country':'Origin Country','Destination  _Destination Country':'Destination Country','Origin  _Origin Gateway':'Origin Gateway','Destination  _Destination Gateway':'Destination Gateway','Origin  _Origin City':'Origin City','Destination  _Destination City':'Destination City'})
    #df1.loc[~df1["Country House"].isnull(), 'Country House?'] = 'Yes'
    #df1['Country House?'].fillna('No', inplace = True)
    df1['Hazardous (Y/N)']=df1['Shipment Information_Hazardous ( Y/N)'].copy()
    df1['General Cargo'] = df1['Hazardous (Y/N)'].copy()
    df1.loc[df1['General Cargo'] == 'N', 'General Cargo'] = 'No'
    df1['General Cargo'] = df1['General Cargo'].str.upper()
    
    #Region to region Table 
    region_to_region = df1.groupby(['Country House?', 'General Cargo', 'Origin Region',
       'Destination Region']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'}, margins = True)
    try:
        region_to_region['% of Chargeable_Weight'] = (region_to_region['Sum of Chargeable_Weight Kgs'] / region_to_region['Sum of Chargeable_Weight Kgs'].sum()) * 100
    except ZeroDivisionError:
        region_to_region['% of Chargeable_Weight'] = 0
    
    region_to_region['% of Chargeable_Weight'] = region_to_region['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        region_to_region['Average weight per shipment (Kgs)'] = (region_to_region['Sum of Chargeable_Weight Kgs'] / (region_to_region['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        region_to_region['Average weight per shipment (Kgs)'] = 0
        
        
    #Country to country Table
    country_to_country_df = df1.groupby(['Country House?', 'General Cargo', 'Origin Country',
       'Destination Country']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    
    try:
        country_to_country_df['% of Chargeable_Weight'] = (country_to_country_df['Sum of Chargeable_Weight Kgs']/ country_to_country_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        country_to_country_df['% of Chargeable_Weight'] = 0
    country_to_country_df['% of Chargeable_Weight'] = country_to_country_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        country_to_country_df['Average weight per shipment (Kgs)'] = (country_to_country_df['Sum of Chargeable_Weight Kgs'] / (country_to_country_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        country_to_country_df['Average weight per shipment (Kgs)'] = 0 
        
        
    #Gateway to gateway
    gateway_to_gateway_df = df1.groupby(['Country House?', 'General Cargo', 'Origin Gateway',
       'Destination Gateway']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        gateway_to_gateway_df['% of Chargeable_Weight'] = (gateway_to_gateway_df['Sum of Chargeable_Weight Kgs']/ gateway_to_gateway_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        gateway_to_gateway_df['% of Chargeable_Weight'] = 0
    gateway_to_gateway_df['% of Chargeable_Weight'] = gateway_to_gateway_df['% of Chargeable_Weight'].apply(lambda x : round(x,2))
    try:
        gateway_to_gateway_df['Average weight per shipment (Kgs)'] = (gateway_to_gateway_df['Sum of Chargeable_Weight Kgs']/(gateway_to_gateway_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        gateway_to_gateway_df['Average weight per shipment (Kgs)'] = 0

    #Origin city to Destination City
    og_city_to_dest_city_df = df1.groupby(['Country House?', 'General Cargo',  'Origin Country',
       'Destination Country','Origin City',
       'Destination City']).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        og_city_to_dest_city_df['% of Chargeable_Weight'] = (og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs']/ og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        og_city_to_dest_city_df['% of Chargeable_Weight'] = 0
    og_city_to_dest_city_df['% of Chargeable_Weight'] = og_city_to_dest_city_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        og_city_to_dest_city_df['Average weight per shipment (Kgs)'] = (og_city_to_dest_city_df['Sum of Chargeable_Weight Kgs']/(og_city_to_dest_city_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        og_city_to_dest_city_df['Average weight per shipment (Kgs)'] = 0 

    #Origin
    origin_df = df1.groupby([  'Origin Country',
      'Origin Gateway','Origin City',
       ]).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    
    try:
        origin_df['% of Chargeable_Weight'] = (origin_df['Sum of Chargeable_Weight Kgs']/ origin_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        origin_df['% of Chargeable_Weight'] = 0 
    origin_df['% of Chargeable_Weight'] = origin_df['% of Chargeable_Weight'].apply(lambda x: round(x, 2))
    try:
        origin_df['Average weight per shipment (Kgs)'] = (origin_df['Sum of Chargeable_Weight Kgs']/(origin_df['No of shipments'] )).astype(int).round()
    except ZeroDivisionError:
        origin_df['Average weight per shipment (Kgs)'] = 0 
        
    #Destination
    destination_df = df1.groupby([  'Destination Country',
      'Destination Gateway','Destination City',
       ]).agg({'No of lanes' : 'count', 'Sum of Chargeable_Weight Kgs' : 'sum', 'No of shipments' : 'sum'})
    try:
        destination_df['% of Chargeable_Weight'] = (destination_df['Sum of Chargeable_Weight Kgs']/ destination_df['Sum of Chargeable_Weight Kgs'].sum())*100
    except ZeroDivisionError:
        destination_df['% of Chargeable_Weight'] = 0
    destination_df['% of Chargeable_Weight'] = destination_df['% of Chargeable_Weight'].apply(lambda x : round(x,2))
    try:
        destination_df['Average weight per shipment (Kgs)'] = (destination_df['Sum of Chargeable_Weight Kgs']/(destination_df['No of shipments'])).astype(int).round()
    except ZeroDivisionError:
        destination_df['Average weight per shipment (Kgs)'] = 0

    with pd.ExcelWriter(save_path) as writer:
        
    # Write each dataframe to a separate sheet
        region_to_region.to_excel(writer, sheet_name='Region to Regrion')
        country_to_country_df.to_excel(writer, sheet_name='Country to Country')
        gateway_to_gateway_df.to_excel(writer, sheet_name='Gateway to Gateway')
        og_city_to_dest_city_df.to_excel(writer, sheet_name='Origin city to Destion city')
        origin_df.to_excel(writer, sheet_name='Origin')
        destination_df.to_excel(writer, sheet_name='Destination')  
        pass


In [10]:
def download_excel():
    file_path = filedialog.asksaveasfilename(defaultextension='.xlsx')
    new_columns = [tuple(col.split('_')) for col in  gcr_data.columns]
    gcr_data.columns = pd.MultiIndex.from_tuples(new_columns)
    gcr_data.dropna(how='all',inplace=True)   
    gcr_data.reset_index(drop=True, inplace=True)
    l=len(gcr_data)
    index=pd.Index(range(l),name='index')
    gcr_data.index=index
    gcr_data.to_excel(file_path)
    print("Excel file saved successfully!")
    pass





In [ ]:
# Create the main window
customtkinter.set_appearance_mode("dark")
window = customtkinter.CTk()
#window.geometry("600x700")
window.title("CustomTkinter simple_example.py")


window.state("zoomed")
window.grid_rowconfigure((0,1,2), weight=1)
window.grid_columnconfigure((0,1,2), weight=1)
tabview = customtkinter.CTkTabview(master=window)
#tabview.grid_rowconfigure((0,1,2), weight=1)
tabview.grid(row=0,sticky='nsew',padx=20,pady=20,rowspan=3,columnspan=3)
tabview.add("tab 1")  # add tab at the end
tabview.add("tab 2")  # add tab at the end
tabview.set("tab 1")  # set currently visible tab

button = customtkinter.CTkButton(master=tabview.tab("tab 2"))
button.grid(row=1,sticky='nsew',padx=20, pady=20)


frame_1 = customtkinter.CTkFrame(master=tabview.tab("tab 1"))
frame_1.grid(row=1,sticky='nsew',padx=20,pady=20)

frame_2 = customtkinter.CTkScrollableFrame(master=tabview.tab("tab 1"), orientation="vertical", label_text="Frame2",width=750, height=460)
frame_2.grid(row=1, column=1, padx=30, pady=20,sticky='nsew',rowspan=4)
frame_3=customtkinter.CTkFrame(master=tabview.tab("tab 1"))
frame_3.grid(row=2,sticky='nsew',padx=20,pady=20)
frame_4=customtkinter.CTkFrame(master=tabview.tab("tab 1"))
frame_4.grid(row=3,sticky='nsew',padx=20,pady=20)



frame_1_ = customtkinter.CTkFrame(master=tabview.tab("tab 2"))
frame_1_.grid(row=1,sticky='nsew',padx=20,pady=20)
frame_2_ = customtkinter.CTkScrollableFrame(master=tabview.tab("tab 2"), orientation="vertical", label_text="Frame2",width=750, height=460)
frame_2_.grid(row=1, column=1, padx=30, pady=20,sticky='nsew',rowspan=4)
frame_3_=customtkinter.CTkFrame(master=tabview.tab("tab 2"))
frame_3_.grid(row=2,sticky='nsew',padx=20,pady=20)
frame_4_=customtkinter.CTkFrame(master=tabview.tab("tab 2"))
frame_4_.grid(row=3,sticky='nsew',padx=20,pady=20)


# Choose Internal Template button
internal_template_button_ = customtkinter.CTkButton(frame_1_, text="Choose Internal Template", command=choose_internal_template,
                                     width=28, anchor="center")
internal_template_button_.grid(row=1, column=0, sticky="w", padx=10, pady=3)

# Choose Customer Template button
customer_template_button_ = customtkinter.CTkButton(frame_1_, text="Choose Customer Template", command=choose_customer_template,
                                     width=28, anchor="center")
customer_template_button_.grid(row=1, column=1, sticky="w", padx=10, pady=3)

# #gcr button
# gcr_template_button_ = customtkinter.CTkButton(frame_1_, text="Choose GCR Template", command=choose_gcr_template,
#                                      width=28, anchor="center")
# gcr_template_button_.grid(row=2, column=0, sticky="w", padx=10, pady=3)

# #gateway Button

# customer_template_button_ = customtkinter.CTkButton(frame_1_, text="Choose Gateway File", command=choose_gateway_file,
#                                      width=28, anchor="center")
# customer_template_button_.grid(row=1, column=2, sticky="w", padx=10, pady=3)

# Label and Entry for number of rows to skip
label_s_entry_ = customtkinter.CTkLabel(frame_3_, text="Number of rows to skip:")
label_s_entry_.grid(row=2, column=0, sticky="w")

skip_rows_entry_ = customtkinter.CTkEntry(frame_3_, width=34)
skip_rows_entry_.grid(row=2, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for number of headers
label_n_entry_ = customtkinter.CTkLabel(frame_3_, text="Number of headers:")
label_n_entry_.grid(row=3, column=0, sticky="w")

num_rows_entry_ = customtkinter.CTkEntry(frame_3_, width=34)
num_rows_entry_.grid(row=3, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for sheet name
label_sheet_entry_ = customtkinter.CTkLabel(frame_3_, text="Sheet Name:")
label_sheet_entry_.grid(row=4, column=0, sticky="w")

sheet_entry_ = customtkinter.CTkEntry(frame_3_, width=34)
sheet_entry_.grid(row=4, column=1, sticky="w", padx=10, pady=3)

# Show Columns button
transfer_button_ = customtkinter.CTkButton(frame_4_, text="Show Columns", command=transfer_data_2, width=28, anchor="center")
transfer_button_.grid(row=7, column=0, sticky="w", padx=10, pady=3)

 # Transfer button for all columns
transfer_all_button = customtkinter.CTkButton(frame_4_, text="Transfer All", command=transfer_all_data_2, width=28, anchor="center")
transfer_all_button.grid(row=7, column=1, sticky="w", padx=10, pady=3)

# # Transfer Gateway Data button
# transfer_all_button = customtkinter.CTkButton(frame_4_, text="Transfer Gateway Data", command=truck_df_treatment, width=28, anchor="center")
# transfer_all_button.grid(row=7, column=2, sticky="w", padx=10, pady=3)

download_button = customtkinter.CTkButton(frame_4_, text="Download Excel", command=download_excel_2, width=28, anchor="center")
download_button.grid(row=8, column=0, sticky="w", padx=10, pady=3)










# heading_l=tk.Label(window,text="Bid Processing App",font=("Arial",16,"bold"))
# heading_l.grid(row=0, column=0, sticky="w", padx=10, pady=3)

# Choose Internal Template button
internal_template_button = customtkinter.CTkButton(frame_1, text="Choose Internal Template", command=choose_internal_template,
                                     width=28, anchor="center")
internal_template_button.grid(row=1, column=0, sticky="w", padx=10, pady=3)

# Choose Customer Template button
customer_template_button = customtkinter.CTkButton(frame_1, text="Choose Customer Template", command=choose_customer_template,
                                     width=28, anchor="center")
customer_template_button.grid(row=1, column=1, sticky="w", padx=10, pady=3)

#gcr button
gcr_template_button = customtkinter.CTkButton(frame_1, text="Choose GCR Template", command=choose_gcr_template,
                                     width=28, anchor="center")
gcr_template_button.grid(row=2, column=0, sticky="w", padx=10, pady=3)

#gateway Button

customer_template_button = customtkinter.CTkButton(frame_1, text="Choose Gateway File", command=choose_gateway_file,
                                     width=28, anchor="center")
customer_template_button.grid(row=1, column=2, sticky="w", padx=10, pady=3)

# Label and Entry for number of rows to skip
label_s_entry = customtkinter.CTkLabel(frame_3, text="Number of rows to skip:")
label_s_entry.grid(row=2, column=0, sticky="w")

skip_rows_entry = customtkinter.CTkEntry(frame_3, width=34)
skip_rows_entry.grid(row=2, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for number of headers
label_n_entry = customtkinter.CTkLabel(frame_3, text="Number of headers:")
label_n_entry.grid(row=3, column=0, sticky="w")

num_rows_entry = customtkinter.CTkEntry(frame_3, width=34)
num_rows_entry.grid(row=3, column=1, sticky="w", padx=10, pady=3)

# Label and Entry for sheet name
label_sheet_entry = customtkinter.CTkLabel(frame_3, text="Sheet Name:")
label_sheet_entry.grid(row=4, column=0, sticky="w")

sheet_entry = customtkinter.CTkEntry(frame_3, width=34)
sheet_entry.grid(row=4, column=1, sticky="w", padx=10, pady=3)

# Show Columns button

transfer_button = customtkinter.CTkButton(frame_4, text="Show Columns", command=transfer_data, width=28, anchor="center")
transfer_button.grid(row=7, column=0, sticky="w", padx=10, pady=3)

 # Transfer button for all columns
transfer_all_button = customtkinter.CTkButton(frame_4, text="Transfer All", command=transfer_all_data, width=28, anchor="center")
transfer_all_button.grid(row=7, column=1, sticky="w", padx=10, pady=3)

# Transfer Gateway Data button
transfer_all_button = customtkinter.CTkButton(frame_4, text="Transfer Gateway Data", command=truck_df_treatment, width=28, anchor="center")
transfer_all_button.grid(row=7, column=2, sticky="w", padx=10, pady=3)

# Download Summary Excel button
download_button = customtkinter.CTkButton(frame_4, text=" Download Summary Table", command=summary_tables, width=28, anchor="center")
download_button.grid(row=7, column=3, sticky="w", padx=10, pady=3)

# Download Excel button
download_button = customtkinter.CTkButton(frame_4, text="Download Excel", command=download_excel, width=28, anchor="center")
download_button.grid(row=8, column=0, sticky="w", padx=10, pady=3)





# Run the main window loop
window.mainloop()


Internal template loaded successfully!


In [ ]:
gcr_data

In [ ]:
internal_template

In [ ]:
processed_data